In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time
tqdm.tqdm.pandas()

In [2]:
bulk_df = pd.read_excel('./공공주택찾기/경기도/일괄계약전체_20180101_20220222.xlsx')
print(bulk_df.shape)
bulk_df.head()

(19561, 15)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약
0,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",50.070,8200,5,2013,2020-03-18,163.770721,9,연립다세대,Y
1,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",57.195,8400,5,2013,2020-03-18,146.865985,9,연립다세대,Y
2,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",58.695,8600,2,2013,2020-03-18,146.520147,9,연립다세대,Y
3,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",58.695,8600,3,2013,2020-03-18,146.520147,9,연립다세대,Y
4,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",58.695,8600,4,2013,2020-03-18,146.520147,9,연립다세대,Y


In [3]:
bulk_df['소유자'] = np.nan

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Get LATEST driver version for 98.0.4758
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\max6296\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [6]:
driver.get("http://www.iros.go.kr/PMainJ.jsp")

In [7]:
driver.get("http://www.iros.go.kr/frontservlet?cmd=RISUWelcomeViewC")

In [8]:
def get_owner_name(df):
    df = df.copy()
    
    count = 0
    for addr in tqdm.tqdm_notebook(df['지번주소'].unique()):
        owner_name = np.nan        
        addr_df = df[df['지번주소'] == addr]
        addr = addr.replace('-0', '')
        
        driver.switch_to.default_content()
        
        
        driver.switch_to.frame("inputFrame")
        
        elem_search = driver.find_element_by_xpath('html/body/form[1]/div[24]/div/div/div/fieldset/div/table/tbody/tr[5]/td/span/input')
        elem_search.clear()
        elem_search.send_keys(addr)
        elem_search.send_keys(Keys.RETURN)
        time.sleep(2)
        
        try:
            driver.switch_to.alert.accept()
        except:
            pass
        
        time.sleep(1)
        
        driver.switch_to.default_content()
        driver.switch_to.frame('resultFrame')
        driver.switch_to.frame('frmOuterModal')
        
        try:
            driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr[3]/td[6]/button').click()
        except:
            continue
        
        time.sleep(1)
        
        elem_result = driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr/td[3]')
        
        owner_name = elem_result.text
        
        driver.switch_to.default_content()
        
        df.loc[addr_df.index, '소유자'] = owner_name
        
        count += 1
    
    return df

In [9]:
prac_df = get_owner_name(bulk_df)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/986 [00:00<?, ?it/s]

In [10]:
print(prac_df.shape)
prac_df.head()

(19561, 16)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약,소유자
0,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",50.070,8200,5,2013,2020-03-18,163.770721,9,연립다세대,Y,주식회사무***~
1,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",57.195,8400,5,2013,2020-03-18,146.865985,9,연립다세대,Y,주식회사무***~
2,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",58.695,8600,2,2013,2020-03-18,146.520147,9,연립다세대,Y,주식회사무***~
3,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",58.695,8600,3,2013,2020-03-18,146.520147,9,연립다세대,Y,주식회사무***~
4,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",58.695,8600,4,2013,2020-03-18,146.520147,9,연립다세대,Y,주식회사무***~


In [11]:
prac_df['소유자'].value_counts()

한***~            7407
김**              1200
이**               740
김**외 1명           540
경***~             521
송**               382
박**               307
조**               286
정**               246
이**외 1명           241
주식회사무***~         196
교***~             196
윤**               163
한**               142
주식회사하***~         132
유**               122
최**               121
장**외 1명           111
조**외 1명           111
고**               102
신**외 1명            98
전**외 1명            98
신***~              94
윤**외 1명            93
안**                92
박**외 1명            88
서**                82
주식회사한***~          80
강**                76
오**                71
신**                70
성**                68
전**                67
홍**                62
주식회사인***           60
백**                60
임**                59
동***~              58
양**                57
유**외 1명            55
우**                54
주식회사신***~          48
지**                48
주식회사아***~          47
주식회사생***~          47
현**       

In [12]:
prac_df['소유자'].isna().sum()

2296

In [13]:
nan_df = prac_df[prac_df['소유자'].isna()]
print(nan_df.shape)
nan_df.head()

(2296, 16)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약,소유자
2385,경기도 부천시 송내동 673-8,성주로132번길 38-10,경기도 부천시 송내동,673,8,희망꿈터,46.36,17882,3,2019,2019-08-26,385.720449,3,연립다세대,Y,NaN
2386,경기도 부천시 송내동 673-8,성주로132번길 38-10,경기도 부천시 송내동,673,8,희망꿈터,50.11,19328,3,2019,2019-08-26,385.711435,3,연립다세대,Y,NaN
2387,경기도 부천시 송내동 673-8,성주로132번길 38-10,경기도 부천시 송내동,673,8,희망꿈터,51.24,19764,3,2019,2019-08-26,385.714286,3,연립다세대,Y,NaN
2388,경기도 부천시 송내동 673-8,성주로132번길 38-10,경기도 부천시 송내동,673,8,희망꿈터,52.66,20311,2,2019,2019-08-26,385.700722,3,연립다세대,Y,NaN
2389,경기도 부천시 송내동 673-8,성주로132번길 38-10,경기도 부천시 송내동,673,8,희망꿈터,55.00,21214,2,2019,2019-08-26,385.709091,3,연립다세대,Y,NaN


In [14]:
prac_df = prac_df.dropna(subset=['소유자']).reset_index(drop=True)

In [15]:
prac_df.shape

(17265, 16)

In [17]:
def sh_lh(x):
    if x == '한***~':
        return 'LH'
    else:
        return x

In [18]:
prac_df['소유자'] = prac_df['소유자'].apply(sh_lh)

In [19]:
prac_df['소유자'].value_counts()

LH               7407
김**              1200
이**               740
김**외 1명           540
경***~             521
송**               382
박**               307
조**               286
정**               246
이**외 1명           241
주식회사무***~         196
교***~             196
윤**               163
한**               142
주식회사하***~         132
유**               122
최**               121
장**외 1명           111
조**외 1명           111
고**               102
신**외 1명            98
전**외 1명            98
신***~              94
윤**외 1명            93
안**                92
박**외 1명            88
서**                82
주식회사한***~          80
강**                76
오**                71
신**                70
성**                68
전**                67
홍**                62
주식회사인***           60
백**                60
임**                59
동***~              58
양**                57
유**외 1명            55
우**                54
주식회사신***~          48
지**                48
주식회사아***~          47
주식회사생***~          47
현**       

In [20]:
%%time
prac_df.to_excel('./공공주택찾기/경기도/일괄계약_전체_소유자포함_20180101_20220222.xlsx', index=False)

Wall time: 5.01 s


In [21]:
prac_df = prac_df[
    (prac_df['소유자'] == 'LH')
]
print(prac_df.shape)
prac_df.head()

(7407, 16)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약,소유자
187,경기도 고양덕양구 성사동 705-18,고양대로1395번길 11-1,경기도 고양덕양구 성사동,705,18,예그리나4차,42.95,29362,3,2021,2021-11-11,683.632130,1,연립다세대,Y,LH
188,경기도 고양덕양구 성사동 705-18,고양대로1395번길 11-1,경기도 고양덕양구 성사동,705,18,예그리나4차,42.95,29362,4,2021,2021-11-11,683.632130,1,연립다세대,Y,LH
189,경기도 고양덕양구 성사동 705-18,고양대로1395번길 11-1,경기도 고양덕양구 성사동,705,18,예그리나4차,42.95,28721,2,2021,2021-11-11,668.707800,1,연립다세대,Y,LH
190,경기도 고양덕양구 성사동 705-18,고양대로1395번길 11-1,경기도 고양덕양구 성사동,705,18,예그리나4차,48.53,31421,3,2021,2021-11-11,647.455182,1,연립다세대,Y,LH
191,경기도 고양덕양구 성사동 705-18,고양대로1395번길 11-1,경기도 고양덕양구 성사동,705,18,예그리나4차,48.53,30715,2,2021,2021-11-11,632.907480,1,연립다세대,Y,LH


In [22]:
prac_df['소유자'].value_counts()

LH    7407
Name: 소유자, dtype: int64

In [23]:
%%time
prac_df.to_excel('./공공주택찾기/경기도/LH_거래사례_20180101_20220222.xlsx', index=False)

Wall time: 2.14 s
